In [22]:
import numpy as np
import pandas as pd

In [23]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [24]:
print(f"The Training set has {train.size} elements and its dimensions are {train.size}")
print(f"The Testing set has {test.size} elements and its dimensions are {test.size}")

The Training set has 95886 elements and its dimensions are 95886
The Testing set has 34394 elements and its dimensions are 34394


In [25]:
import re
def clean_text(df,text_field):
    df[text_field]=df[text_field].str.lower()
    df[text_field]=df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "",elem))
    return df
test_clean=clean_text(test,"tweet")
train_clean=clean_text(train,"tweet")

In [26]:
from sklearn.utils import resample
train_majority=train_clean[train_clean.label==0]
train_minority=train_clean[train_clean.label==1]
train_minority_upsampled=resample(train_minority,replace=True,n_samples=len(train_majority),random_state=123)
train_upsampled=pd.concat([train_minority_upsampled,train_majority])
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd=Pipeline([('vect',CountVectorizer()),('tfidf',TfidfTransformer()),('nb',SGDClassifier()),])

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_upsampled['tweet'],train_upsampled['label'],random_state=0)

In [29]:
model=pipeline_sgd.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [30]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
ascore=accuracy_score(y_test,y_pred)
f1s=f1_score(y_test,y_pred)
cr=classification_report(y_test,y_pred)
print(f"The Accuracy score of our model is {round(f1s*100)}%.\n")
print(f"The F1 score of our model is {round(f1s*100)}%.\n")
print(f"The Classification Report of our model is as follows\n",cr)

The Accuracy score of our model is 97%.

The F1 score of our model is 97%.

The Classification Report of our model is as follows
               precision    recall  f1-score   support

           0       0.99      0.95      0.97      7490
           1       0.95      0.99      0.97      7370

    accuracy                           0.97     14860
   macro avg       0.97      0.97      0.97     14860
weighted avg       0.97      0.97      0.97     14860

